In [70]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
import pathlib
data_root_orig = tf.keras.utils.get_file(origin='https://commondatastorage.googleapis.com/books1000/notMNIST_small.tar.gz',
                                         fname='notMNIST_small', untar=True)
data_root = pathlib.Path(data_root_orig)
print(data_root)

/home/maxim/.keras/datasets/notMNIST_small


In [71]:
for letter_dir in data_root.iterdir():
  print(letter_dir)

/home/maxim/.keras/datasets/notMNIST_small/F
/home/maxim/.keras/datasets/notMNIST_small/B
/home/maxim/.keras/datasets/notMNIST_small/H
/home/maxim/.keras/datasets/notMNIST_small/G
/home/maxim/.keras/datasets/notMNIST_small/D
/home/maxim/.keras/datasets/notMNIST_small/I
/home/maxim/.keras/datasets/notMNIST_small/C
/home/maxim/.keras/datasets/notMNIST_small/A
/home/maxim/.keras/datasets/notMNIST_small/J
/home/maxim/.keras/datasets/notMNIST_small/E


In [72]:
import random
all_image_paths = list(data_root.glob('*/*'))
all_image_paths = [str(path) for path in all_image_paths]
random.shuffle(all_image_paths)

image_count = len(all_image_paths)
image_count

18726

In [73]:
all_image_paths[:10]

import IPython.display as display

for n in range(5):
  image_path = random.choice(all_image_paths)
  display.display(display.Image(image_path))
  print(image_path)

/home/maxim/.keras/datasets/notMNIST_small/A/Q29vcGVyQlQtQmxhY2tIZWFkbGluZS5vdGY=.png


/home/maxim/.keras/datasets/notMNIST_small/F/R2lsbFNhbnNNVFN0ZC1Cb2xkLm90Zg==.png


/home/maxim/.keras/datasets/notMNIST_small/C/QXVyaW9sTFRTdGQtQmxhY2tJdGFsaWMub3Rm.png


/home/maxim/.keras/datasets/notMNIST_small/G/RWxhbkVGLUJsYWNrLm90Zg==.png


/home/maxim/.keras/datasets/notMNIST_small/B/QW50aXF1YSAxMDEgQ29uZGVuc2VkIEl0YWxpYy50dGY=.png


In [74]:
label_names = sorted(item.name for item in data_root.glob('*/') if item.is_dir())
label_names

['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J']

In [75]:
label_to_index = dict((name, index) for index, name in enumerate(label_names))
label_to_index

{'A': 0,
 'B': 1,
 'C': 2,
 'D': 3,
 'E': 4,
 'F': 5,
 'G': 6,
 'H': 7,
 'I': 8,
 'J': 9}

In [76]:
all_image_labels = [label_to_index[pathlib.Path(path).parent.name]
                    for path in all_image_paths]

print("First 10 labels indices: ", all_image_labels[:10])
print("First 10 paths: ", all_image_paths[:10])

First 10 labels indices:  [4, 1, 7, 6, 1, 7, 6, 1, 4, 4]
First 10 paths:  ['/home/maxim/.keras/datasets/notMNIST_small/E/RGFudGVNVFN0ZC1SZWd1bGFyLm90Zg==.png', '/home/maxim/.keras/datasets/notMNIST_small/B/QXJjaGUgQmxhY2sgQ29uZGVuc2VkIFNTaSBCbGFjayBDb25kZW5zZWQudHRm.png', '/home/maxim/.keras/datasets/notMNIST_small/H/Q2FydGllckJvb2tTdGQtTWVkaXVtLm90Zg==.png', '/home/maxim/.keras/datasets/notMNIST_small/G/SG9ybnB5cGVJVENTdGQub3Rm.png', '/home/maxim/.keras/datasets/notMNIST_small/B/SG9uZXlib21iLnR0Zg==.png', '/home/maxim/.keras/datasets/notMNIST_small/H/QmVya2VsZXlPbGRTdHlsZUVGLUJvbGRJdGFsaWMub3Rm.png', '/home/maxim/.keras/datasets/notMNIST_small/G/R291ZHlDYXRhbG9ndWVFRi1SZWd1bGFyLm90Zg==.png', '/home/maxim/.keras/datasets/notMNIST_small/B/RnJlaWdodEJpZ0xpZ2h0SXRhbGljU0MudHRm.png', '/home/maxim/.keras/datasets/notMNIST_small/E/RHVzdGlzbW8gIEJvbGQgSXRhbGljLnR0Zg==.png', '/home/maxim/.keras/datasets/notMNIST_small/E/SHVtYW5pc3QgOTcwIEJvbGQucGZi.png']


In [77]:
img_path = all_image_paths[0]
img_path
img_raw = tf.io.read_file(img_path)
print(repr(img_raw)[:100]+"...")

img_tensor = tf.image.decode_image(img_raw)

print(img_tensor.shape)
print(img_tensor.dtype)

<tf.Tensor: shape=(), dtype=string, numpy=b"\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x00\x1c\x00\...
(28, 28, 1)
<dtype: 'uint8'>


In [78]:
def preprocess_image(image):
  image = tf.image.decode_jpeg(image, channels=1)
  image = tf.image.resize(image, [28, 28])
  return image

def load_and_preprocess_image(path):
  image = tf.io.read_file(path)
  return preprocess_image(image)

In [79]:
path_ds = tf.data.Dataset.from_tensor_slices(all_image_paths)
print(path_ds)

<TensorSliceDataset shapes: (), types: tf.string>


In [80]:
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=tf.data.experimental.AUTOTUNE)
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(all_image_labels, tf.int64))

for label in label_ds.take(10):
  print(label_names[label.numpy()])

print(label_ds)

image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))
print(image_label_ds)

E
B
H
G
B
H
G
B
E
E
<TensorSliceDataset shapes: (), types: tf.int64>
<ZipDataset shapes: ((28, 28, 1), ()), types: (tf.float32, tf.int64)>


In [86]:
BATCH_SIZE = 32

# Установка размера буфера перемешивания, равного набору данных, гарантирует
# полное перемешивание данных.
ds = image_label_ds.shuffle(buffer_size=image_count)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` позволяет датасету извлекать пакеты в фоновом режиме, во время обучения модели.
ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

ds = image_label_ds.apply(
  tf.data.experimental.shuffle_and_repeat(buffer_size=image_count))
ds = ds.batch(BATCH_SIZE)
ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
ds

<PrefetchDataset shapes: ((None, 28, 28, 1), (None,)), types: (tf.float32, tf.int64)>

In [90]:
from tensorflow import keras
model = keras.Sequential([
            keras.layers.Input(shape=(28, 28, 1)),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(128, activation='relu'),
            keras.layers.Dense(10, activation='softmax')
        ])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(ds, steps_per_epoch=100)

Train for 100 steps
  1/100 [..............................] - ETA: 40s

InvalidArgumentError:  Expected image (JPEG, PNG, or GIF), got empty file
	 [[{{node DecodeJpeg}}]]
	 [[IteratorGetNext]] [Op:__inference_distributed_function_4171]

Function call stack:
distributed_function
